# Hand Tracking and Left/Right Hand Detection

In [ ]:
#Make sure the python version is 3.8 - 3.11
!python --version 

In [6]:
!pip install mediapipe opencv-python

In [99]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [107]:
def get_label(index, hand, results):
    output = None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            
            # Process results
            label = classification.classification[0].label
            score = classification.classification[0].score
            text = '{} {}'.format(label, round(score, 2))
            
            # Extract Coordinates
            coords = tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x, hand.landmark[mp_hands.HandLandmark.WRIST].y)),
            [640,480]).astype(int))
            
            output = text, coords
            
    return output

In [116]:
get_label(num,hand,results)

('Left 0.97', (214, 341))

In [115]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        
        
        # Detections
        # print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),)

                if get_label(num, hand, results):
                    text, coord = get_label(num, hand, results)
                    cv2.putText(image, text, coord, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [24]:
for x in results.multi_hand_landmarks:
    if (x.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]).y > (x.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP]).y:
        print("index finger down")
    else:
        print("index finger up")

index finger up


In [129]:
results.multi_handedness[0].classification[0]

index: 0
score: 0.97322893
label: "Left"

In [60]:
results.multi_handedness

[classification {
   index: 0
   score: 0.9778062
   label: "Left"
 },
 classification {
   index: 1
   score: 0.8947927
   label: "Right"
 }]

In [123]:
results.multi_hand_landmarks[0].landmark[20]

x: 0.31693545
y: 0.46262664
z: -0.0385514